# DeepFake Face Detector Training Notebook
This notebook trains a computer to spot fake images using a Vision Transformer (ViT) model.

## 1. Importing Libraries
Python makes it easy to use powerful libraries for machine learning and AI. Here, we import:
- `os`: For file and directory operations.
- `torch`: The core library for deep learning in Python (PyTorch).
- `transformers`: Provides pre-trained models and tools for AI tasks.
- `datasets`: For loading and managing datasets.
- `sklearn.metrics`: For evaluating model performance.
- `PIL.Image`: For image processing.

These libraries are widely used in the AI community and make complex tasks much simpler.

In [ ]:
# Import required libraries
import os
import torch
from transformers import (
    AutoImageProcessor,
    ViTForImageClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score, classification_report
from PIL import Image

## Configuration & Model Selection
Set up the model checkpoint, image size, and data paths.

## 2. Configuration & Model Selection
We define important settings:
- **Model Checkpoint**: Specifies which pre-trained AI model to use (Vision Transformer).
- **Image Resolution**: Images are resized to 224x224 pixels, matching the model's requirements.
- **Data Path**: Location of the dataset on disk.
- **Output Directory**: Where results and trained models will be saved.

This step shows how Python variables are used to configure experiments and control workflow.

In [ ]:
CHECKPOINT = 'google/vit-base-patch16-224-in21k'
TARGET_RESOLUTION = 224
DATA_PATH = 'E:/data_02'  # Path to dataset with 'train', 'validation', 'test' folders
OUTPUT_DIR = 'E:/vit_in21k_output_02'

## Load Data
Load the dataset from the specified folder.

## 3. Loading the Dataset
We use the `datasets` library to load images from folders. Python makes it easy to:
- Automatically detect training, validation, and test sets.
- Organize data for machine learning tasks.

This step demonstrates how AI projects start with data collection and preparation.

In [ ]:
print(f'Loading data from: {DATA_PATH}')
dataset = load_dataset('imagefolder', data_dir=DATA_PATH)

## Prepare Data
Get label names and mappings.

## 4. Label Mapping
Machine learning models need numbers, not words. We:
- Extract label names (like 'real', 'fake').
- Convert them to numbers so the model can understand them.

This is a key step in supervised learning, where each example has a known answer.

In [ ]:
label_names = dataset['train'].features['label'].names
num_labels = len(label_names)
label2id = {name: i for i, name in enumerate(label_names)}
id2label = {i: name for i, name in enumerate(label_names)}
print(f'Detected Labels ({num_labels}): {id2label}')

## Load Processor and Model
Load the image processor and the Vision Transformer model.

## 5. Model and Processor Initialization
We use a pre-trained Vision Transformer (ViT) model, a powerful AI that understands images. Python lets us:
- Load the model and processor with one line of code.
- Adapt the model for our specific task (number of labels).

This step shows transfer learning, where we use knowledge from a model trained on millions of images.

In [ ]:
processor = AutoImageProcessor.from_pretrained(CHECKPOINT)
model = ViTForImageClassification.from_pretrained(
    CHECKPOINT,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    problem_type='single_label_classification'
)

## Preprocess Function
Resize images and prepare them for the model.

## 6. Preprocessing Images
AI models need images in a specific format. We:
- Resize images to the right size (224x224 pixels).
- Convert them to tensors (arrays of numbers) for the model.

Python functions make it easy to automate this for every image.

In [ ]:
def preprocess(example):
    image = example['image'].convert('RGB').resize((TARGET_RESOLUTION, TARGET_RESOLUTION))
    inputs = processor(images=image, return_tensors='pt')
    example['pixel_values'] = inputs['pixel_values'][0]
    return example

## Apply Preprocessing and Format Setting
Process all images and set the dataset format for training.

## 7. Dataset Preprocessing
We apply our preprocessing function to every image in the dataset. Python's `map` function makes this efficient and fast.
- The dataset is now ready for training.
- We set the format so PyTorch can use it directly.

This step is crucial for preparing data for machine learning.

In [ ]:
dataset = dataset.map(preprocess, batched=False)
dataset.set_format(type='torch', columns=['pixel_values', 'label'])

## Training Setup
Define metrics and training arguments.

## 8. Metrics Definition
To measure how well our AI is learning, we use metrics:
- **Accuracy**: How many predictions are correct.
- **F1 Score**: Balances accuracy for each class, useful for uneven data.

Python functions let us calculate these automatically during training and testing.

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted')
    }

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=10,
    learning_rate=5e-5,
    logging_dir='./logs',
    logging_steps=10,
    remove_unused_columns=False,
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    max_grad_norm=1.0,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1
)

## Training Arguments
Set up how the model will learn.

## 9. Training Arguments Setup
We configure how the AI will learn:
- **Batch Size**: Number of images processed at once.
- **Epochs**: How many times the AI sees the whole dataset.
- **Learning Rate**: How quickly the AI updates its knowledge.
- **Early Stopping**: Stops training if no improvement is seen.

Python makes it easy to set these parameters for experiments.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

## Define Trainer
Set up the Trainer with model, arguments, datasets, and callbacks.

## 10. Trainer Initialization
The `Trainer` class from Hugging Face simplifies training:
- Handles the training loop, evaluation, and saving.
- Uses our model, data, metrics, and settings.
- Supports callbacks like early stopping.

This is a great example of how Python and modern libraries make AI development accessible.

In [ ]:
print('Starting training...')
trainer.train()
trainer.save_model(f'{OUTPUT_DIR}/final_model')
processor.save_pretrained(f'{OUTPUT_DIR}/final_model')
print('Training complete and best model saved.')

## Train the Model
Start training and save the best model.

## 11. Model Training
We start the training process:
- The AI model learns from the training images.
- It checks its progress on the validation set.
- The best version is saved for future use.

This step is where the AI actually learns to spot fake images, using Python code to control the process.

In [ ]:
if 'test' in dataset:
    print('Running final evaluation on the Test set...')
    predictions = trainer.predict(dataset['test'])
    metrics = compute_metrics(predictions)
    print('Test Metrics:', metrics)
    y_true = predictions.label_ids
    y_pred = predictions.predictions.argmax(-1)
    target_names = list(id2label.values())
    print('--- Classification Report ---')
    print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
else:
    print("Warning: 'test' split not found in data. Skipping final evaluation.")
print('Script finished.')

## Final Evaluation on Test Set
Evaluate the model on the test set and print results.

## 12. Model Saving
After training, we save the model and processor:
- This lets us reuse the AI without retraining.
- Python makes saving and loading models simple.

Saving is important for deploying AI in real-world applications.

## 13. Final Evaluation
We test the AI on new images it hasn't seen before:
- Predicts if each image is real or fake.
- Calculates accuracy and F1 score.
- Prints a detailed report.

This step shows how to measure AI performance and ensure it works well in practice.